# Fine tuning de modelos de BERT
En este segundo enfoque vamos a ajustar modelos preentrenados de BERT con Pytorch haciendo uso de la clase Trainer con la que es mucho más sencillo hacer el entrenamiento. Otra opción sería emplear Pytorch en su forma nativa, teniendo que crear los bucles para entrenar y evaluar el modelo manualmente.

En primer lugar, importamos las librerías necesarias para el desarrollo del Notebook.

In [89]:
# Si no tenemos instalada la librería Transformers:
!pip3 install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset   
# Leemos el conjunto de datos 
df = pd.read_csv("data_web/economy-dataset.csv")
df = df.dropna()

label_dict = {
    -1.0 : 0,
    0.0 : 1,
    1.0 : 2,
}
df["S. Target"] = df["S. Target"].replace(label_dict).astype(int)
df["S. Others"] = df["S. Others"].replace(label_dict).astype(int)
df["S. Society"] = df["S. Society"].replace(label_dict).astype(int)
 
# Seleccionamos la dimensión del sentimiento del Target
df["label"] = df["S. Society"]
# Si quisiéramos clasificar el sentimiento de otra dimensión modificaríamos 
# la columna 'label' por la dimensión correspondiente

p_train = 0.80 # Porcentaje de train
p_eval = 0.20 # Porcentaje de eval
p_test = 0.20 # Porcentaje de test

# Mezclamos el dataset
random_state = 8
df = df.sample(frac=1, random_state=random_state)

df_train, df_test = train_test_split (df, test_size = p_test, random_state = random_state)
df_train, df_test = train_test_split (df, test_size = p_test, random_state = random_state)
df_train, df_eval = train_test_split (df_train, test_size = p_eval, random_state = random_state)

dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)
dataset_test = Dataset.from_pandas(df_test)

print("Ejemplos usados para entrenar: ", len(df_train))
print("Ejemplos usados para evaluar: ", len(df_eval))
print("Ejemplos usados para test: ", len(df_test))

Ejemplos usados para entrenar:  1452
Ejemplos usados para evaluar:  364
Ejemplos usados para test:  454


## BERTIN
Comenzamos cargando el Tokenizer y el modelo preentrenado, y leemos el conjunto de datos, para el que hacemos la partición en conjunto de entrenamiento, evaluación y test. El primer modelo con el que ajustaremos nuestro conjunto es BERTIN.




In [91]:
import numpy as np
import json
import torch
from sklearn.metrics import recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer                

In [92]:
def tokenize(batch):
  return tokenizer(batch["Heading"], truncation=True, padding = True)

In [93]:
from functools import partial
path_bert_model = "bertin-project/bertin-roberta-base-spanish" # BERTIN

# Cargamos el Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

dataset_train = dataset_train.map(partial(tokenize), batched = True, batch_size = len(dataset_train))
dataset_eval = dataset_eval.map(partial(tokenize), batched = True, batch_size = len(dataset_eval))
dataset_test = dataset_test.map(partial(tokenize), batched = True, batch_size = len(dataset_test))

loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/f47efb87887425ef9a4ef795bfaa907d57ac9a650d733c7ca621b9eced3235e8.0b968504b15b2f5e6e9e491723f31987782cb976064aff9a7c6188cc2d5ce8bc
loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/aba9e0895dea47dd4208a36012ffd3eb21eb4c5f7ce0be6547afb37cdd4ddef4.0d24ae8bd5fabb1f5020f91bc602cefeb5a2938ab77e21769d28776345634b23
loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/baad57d0f574d3e660cafb14601d0ecebe83f25071d59f3e51d225d75285b773.23862d4cf295978aaf1f42b721fda2d8ece45b112d21f0c83905e5b05454e440
loading file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/added_tokens.json from cache at None
loading file

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [94]:
dataset_train.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_eval.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_test.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)

In [95]:
NUM_LABELS = 3
# Cargamos el modelo para clasificación en Pytorch

bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, 
                                                                              num_labels=NUM_LABELS)

loading configuration file https://huggingface.co/bertin-project/bertin-roberta-base-spanish/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/aa24361f0b7bed62876f6cd0a784a2b622c1959523906d89eeb1112139a4864a.604375914a974f33506fc8902fe2d195407cbf2316166e695267dd5655a736b3
Model config RobertaConfig {
  "_name_or_path": "bertin-project/bertin-roberta-base-spanish",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "n

Preprocesemos el texto usando el Tokenizer inicializado anteriormente. El texto de entrada que estamos usando para el tokenizador es una lista de cadenas.

Ahora definimos  los parámetros de entrenamiento y configuramos el modelo preentrenado en las clases TrainingArgs y Trainer para poder entrenar el modelo con un solo comando. 

Primero definimos una función para calcular las métricas del conjunto de validación. Dado que este es un problema de clasificación binaria con clases desbalanceadas vamos a usar métricas que tengan más en cuenta la clase minoritaria como son la precisión, recall o el F1-score. Una vez que hayamos definido los parámetros, simplemente ejecutamos trainer.train() para entrenar el modelo.

In [96]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    pred = np.argmax(logits, axis=1)

    recall = recall_score(y_true=labels, y_pred=pred, average = "weighted")
    precision = precision_score(y_true=labels, y_pred=pred, average = "weighted")
    f1 = f1_score(y_true=labels, y_pred=pred, average = "weighted")

    return {"Precision": precision, "Recall": recall, "F1": f1}

In [97]:
# Definimos los argumentos para Trainer
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir = './logs',
    evaluation_strategy= "epoch",
    logging_strategy = "epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    seed = 123
)

trainer = Trainer(
    model=bert_class_model_pytorch,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    compute_metrics=compute_metrics
)

# Entrenamos el modelo
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,1.029000,0.997387,0.410138,0.472527,0.428804
2,0.865100,0.920675,0.607098,0.541209,0.550930
3,0.566700,0.928728,0.628069,0.623626,0.624435


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Soc

TrainOutput(global_step=273, training_loss=0.8202638032235505, metrics={'train_runtime': 53.8455, 'train_samples_per_second': 80.898, 'train_steps_per_second': 5.07, 'total_flos': 109687461588936.0, 'train_loss': 0.8202638032235505, 'epoch': 3.0})

In [98]:
print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval()
print(json.dumps(trainer.evaluate(), indent = 2))

print("PREDICCIONES SOBRE TEST")
predictions = trainer.predict(dataset_test)
print(json.dumps(predictions.metrics, indent = 2))

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64


PREDICCIONES SOBRE EVAL


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 454
  Batch size = 64


{
  "eval_loss": 0.9287282824516296,
  "eval_Precision": 0.6280693800278901,
  "eval_Recall": 0.6236263736263736,
  "eval_F1": 0.6244352155031595,
  "eval_runtime": 0.9566,
  "eval_samples_per_second": 380.531,
  "eval_steps_per_second": 6.272,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.9199604988098145,
  "test_Precision": 0.6108909284198675,
  "test_Recall": 0.6123348017621145,
  "test_F1": 0.6085812982993225,
  "test_runtime": 1.1461,
  "test_samples_per_second": 396.121,
  "test_steps_per_second": 6.98
}


## RoBERTa-BNE
Repetimos el proceso para RoBERTa-BNE.

In [99]:
path_bert_model = "PlanTL-GOB-ES/roberta-base-bne" # RoBERTa-BNE

# Cargamos el Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(partial(tokenize), batched = True, batch_size = len(dataset_train))
dataset_eval = dataset_eval.map(partial(tokenize), batched = True, batch_size = len(dataset_eval))
dataset_test = dataset_test.map(partial(tokenize), batched = True, batch_size = len(dataset_test))

loading configuration file https://huggingface.co/PlanTL-GOB-ES/roberta-base-bne/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9559bd682b1ae9cf25eb8bed5a9ab64b481da43e670debc0b519981fea4afd13.33b0b03a5bf5e640494a22a3aa4909c661effc0fa0e186b1513b17d9b058ca59
Model config RobertaConfig {
  "_name_or_path": "PlanTL-GOB-ES/roberta-base-bne",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_siz

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [100]:
dataset_train.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_eval.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_test.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)

In [101]:
NUM_LABELS = 3
# Cargamos el modelo para clasificación en Pytorch

bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, 
                                                                              num_labels=NUM_LABELS)

loading configuration file https://huggingface.co/PlanTL-GOB-ES/roberta-base-bne/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/9559bd682b1ae9cf25eb8bed5a9ab64b481da43e670debc0b519981fea4afd13.33b0b03a5bf5e640494a22a3aa4909c661effc0fa0e186b1513b17d9b058ca59
Model config RobertaConfig {
  "_name_or_path": "PlanTL-GOB-ES/roberta-base-bne",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
 

In [102]:
# Definimos los argumentos para Trainer
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir = './logs',
    evaluation_strategy= "epoch",
    logging_strategy = "epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    seed = 123
)

trainer = Trainer(
    model=bert_class_model_pytorch,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    compute_metrics=compute_metrics
)

# Entrenamos el modelo
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.835400,0.749224,0.712404,0.673077,0.677284
2,0.391400,0.744989,0.723408,0.703297,0.702281
3,0.109400,0.982869,0.712044,0.711538,0.710297


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
The following columns in the evaluation set don't have a cor

TrainOutput(global_step=273, training_loss=0.44539124974401006, metrics={'train_runtime': 50.4832, 'train_samples_per_second': 86.286, 'train_steps_per_second': 5.408, 'total_flos': 109687461588936.0, 'train_loss': 0.44539124974401006, 'epoch': 3.0})

In [103]:
print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval()
print(json.dumps(trainer.evaluate(), indent = 2))

print("PREDICCIONES SOBRE TEST")
predictions = trainer.predict(dataset_test)
print(json.dumps(predictions.metrics, indent = 2))

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64


PREDICCIONES SOBRE EVAL


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 454
  Batch size = 64


{
  "eval_loss": 0.9828689098358154,
  "eval_Precision": 0.7120435732970222,
  "eval_Recall": 0.7115384615384616,
  "eval_F1": 0.7102968042817667,
  "eval_runtime": 0.9713,
  "eval_samples_per_second": 374.75,
  "eval_steps_per_second": 6.177,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.989628255367279,
  "test_Precision": 0.7046005824273078,
  "test_Recall": 0.7026431718061674,
  "test_F1": 0.7026298215108191,
  "test_runtime": 1.148,
  "test_samples_per_second": 395.487,
  "test_steps_per_second": 6.969
}


## BETO

In [104]:
path_bert_model = 'dccuchile/bert-base-spanish-wwm-uncased' # BETO

# Cargamos el Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(partial(tokenize), batched = True, batch_size = len(dataset_train))
dataset_eval = dataset_eval.map(partial(tokenize), batched = True, batch_size = len(dataset_eval))
dataset_test = dataset_test.map(partial(tokenize), batched = True, batch_size = len(dataset_test))

loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [105]:
dataset_train.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_eval.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_test.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)

In [106]:
NUM_LABELS = 3
# Cargamos el modelo para clasificación en Pytorch

bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, 
                                                                              num_labels=NUM_LABELS)

loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_toke

In [107]:
# Definimos los argumentos para Trainer
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir = './logs',
    evaluation_strategy= "epoch",
    logging_strategy = "epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    seed = 123
)

trainer = Trainer(
    model=bert_class_model_pytorch,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    compute_metrics=compute_metrics
)

# Entrenamos el modelo
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
****

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.913500,0.789501,0.637161,0.629121,0.631141
2,0.490700,0.864508,0.653255,0.651099,0.648429
3,0.163900,1.183367,0.650977,0.648352,0.648325


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
The following columns in the evaluation set don't have a corresponding a

TrainOutput(global_step=273, training_loss=0.5227109503833366, metrics={'train_runtime': 54.3568, 'train_samples_per_second': 80.137, 'train_steps_per_second': 5.022, 'total_flos': 116403020461728.0, 'train_loss': 0.5227109503833366, 'epoch': 3.0})

In [108]:
print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval()
print(json.dumps(trainer.evaluate(), indent = 2))


print("PREDICCIONES SOBRE TEST")
predictions = trainer.predict(dataset_test)
print(json.dumps(predictions.metrics, indent = 2))

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64


PREDICCIONES SOBRE EVAL


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 454
  Batch size = 64


{
  "eval_loss": 1.1833665370941162,
  "eval_Precision": 0.650976519270712,
  "eval_Recall": 0.6483516483516484,
  "eval_F1": 0.6483253110307805,
  "eval_runtime": 0.879,
  "eval_samples_per_second": 414.108,
  "eval_steps_per_second": 6.826,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 1.1636203527450562,
  "test_Precision": 0.6715440563652536,
  "test_Recall": 0.6674008810572687,
  "test_F1": 0.6676987965899233,
  "test_runtime": 1.2877,
  "test_samples_per_second": 352.579,
  "test_steps_per_second": 6.213
}


## RoBERTuito

In [109]:
path_bert_model = "pysentimiento/robertuito-base-cased" # RoBERTuito

# Cargamos el Tokenizer 
tokenizer = AutoTokenizer.from_pretrained(path_bert_model)

dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(partial(tokenize), batched = True, batch_size = len(dataset_train))
dataset_eval = dataset_eval.map(partial(tokenize), batched = True, batch_size = len(dataset_eval))
dataset_test = dataset_test.map(partial(tokenize), batched = True, batch_size = len(dataset_test))

loading file https://huggingface.co/pysentimiento/robertuito-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/cc4f40c3fa7221828413091d1002dd792360afc79baa7e0021ca004290bca697.3add50879ecd23ec84a56d45b9389efbefc57b7c9a1deb5a9936e145217a29e7
loading file https://huggingface.co/pysentimiento/robertuito-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/pysentimiento/robertuito-base-cased/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/3be5f21f2981c0ec8d9f3a8644c4c8857d3ab20848cf2fca6ca0e09b4583ba4e.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
loading file https://huggingface.co/pysentimiento/robertuito-base-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/339c5812eaea8622873dfd17d0ff80c36d9b539a9cd2868e30f0516b3de132ff.6ae2760214eca2a5e9eca95c193833cf24e65d6a29ba05cbefbdf22e54a3597d


  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [110]:
dataset_train.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_eval.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)
dataset_test.set_format("torch", columns = ["input_ids", "attention_mask", "label"], output_all_columns= False)

In [111]:
NUM_LABELS = 3
# Cargamos el modelo para clasificación en Pytorch

bert_class_model_pytorch = AutoModelForSequenceClassification.from_pretrained(path_bert_model, 
                                                                              num_labels=NUM_LABELS)

loading configuration file https://huggingface.co/pysentimiento/robertuito-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3f85c0ee804baf604258892a88dd52cdf051d2418a511dcab7cab99a85a3a1b3.4cce50d5a926bf18fe43f2ea8d4596b505e97a64e6e700e993def66b06f1c83b
Model config RobertaConfig {
  "_name_or_path": "pysentimiento/robertuito-base-cased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_laye

In [112]:
# Definimos los argumentos para Trainer
training_args = TrainingArguments(
    output_dir="./results",
    logging_dir = './logs',
    evaluation_strategy = "epoch",
    logging_strategy = "epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    seed = 123
)

trainer = Trainer(
    model=bert_class_model_pytorch,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    compute_metrics=compute_metrics
)

# Entrenamos el modelo
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.883000,0.783136,0.687679,0.651099,0.655392
2,0.496400,0.777781,0.689466,0.684066,0.685915
3,0.244900,0.841328,0.700714,0.700549,0.700625


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64
The following columns in the evaluation set don't have a cor

TrainOutput(global_step=273, training_loss=0.5414564740526807, metrics={'train_runtime': 50.6455, 'train_samples_per_second': 86.01, 'train_steps_per_second': 5.39, 'total_flos': 116403020461728.0, 'train_loss': 0.5414564740526807, 'epoch': 3.0})

In [113]:
print ("PREDICCIONES SOBRE EVAL")
bert_class_model_pytorch.eval()
print(json.dumps(trainer.evaluate(), indent = 2))

print("PREDICCIONES SOBRE TEST")
predictions = trainer.predict(dataset_test)
print(json.dumps(predictions.metrics, indent = 2))

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 364
  Batch size = 64


PREDICCIONES SOBRE EVAL


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading. If S. Society, ID, S. Others, Target, S. Target, __index_level_0__, Heading are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 454
  Batch size = 64


{
  "eval_loss": 0.8413282036781311,
  "eval_Precision": 0.7007143458127872,
  "eval_Recall": 0.7005494505494505,
  "eval_F1": 0.7006252368321334,
  "eval_runtime": 0.8481,
  "eval_samples_per_second": 429.171,
  "eval_steps_per_second": 7.074,
  "epoch": 3.0
}
PREDICCIONES SOBRE TEST
{
  "test_loss": 0.9267518520355225,
  "test_Precision": 0.6472397892064319,
  "test_Recall": 0.6475770925110133,
  "test_F1": 0.6473654410962105,
  "test_runtime": 1.0426,
  "test_samples_per_second": 435.466,
  "test_steps_per_second": 7.673
}
